# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer. The convienient `PredictorInterface.adjust()` does this automatically for you.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

INFO:lightwood-2617:No torchvision detected, image helpers not supported.
INFO:lightwood-2617:No torchvision/pillow detected, image encoder not supported


In [2]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.1*len(df))]
update_df = df[int(0.1*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Train dataframe shape: (103, 10)
Update dataframe shape: (721, 10)
Test dataframe shape: (206, 10)


Note that we have three different data splits.

We will use the `training` split for the initial model training. As you can see, it's only a 20% of the total data we have. The `update` split will be used as training data to adjust/update our model. Finally, the held out `test` set will give us a rough idea of the impact our updating procedure has on the model's predictive capabilities.

In [3]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

# We will keep the architecture simple: a single neural mixer, and a `BestOf` ensemble:
jai.model = {
    "module": "BestOf",
    "args": {
        "args": "$pred_args",
        "accuracy_functions": "$accuracy_functions",
        "submodels": [{
            "module": "Neural",
            "args": {
                "fit_on_dev": False,
                "stop_after": "$problem_definition.seconds_per_mixer",
                "search_hyperparameters": False,
            }
        }]
    }
}

# Build and train the predictor
predictor = predictor_from_json_ai(jai)
predictor.learn(train_df)

INFO:type_infer-2617:Analyzing a sample of 979
INFO:type_infer-2617:from a total population of 1030, this is equivalent to 95.0% of your data.
INFO:type_infer-2617:Infering type for: id
INFO:type_infer-2617:Column id has data type integer
INFO:type_infer-2617:Infering type for: cement
INFO:type_infer-2617:Column cement has data type float
INFO:type_infer-2617:Infering type for: slag
INFO:type_infer-2617:Column slag has data type float
INFO:type_infer-2617:Infering type for: flyAsh
INFO:type_infer-2617:Column flyAsh has data type float
INFO:type_infer-2617:Infering type for: water
INFO:type_infer-2617:Column water has data type float
INFO:type_infer-2617:Infering type for: superPlasticizer
INFO:type_infer-2617:Column superPlasticizer has data type float
INFO:type_infer-2617:Infering type for: coarseAggregate
INFO:type_infer-2617:Column coarseAggregate has data type float
INFO:type_infer-2617:Infering type for: fineAggregate
INFO:type_infer-2617:Column fineAggregate has data type float
I

In [4]:
# Train and get predictions for the held out test set
predictions = predictor.predict(test_df)
predictions

INFO:dataprep_ml-2617:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2617:Cleaning the data
DEBUG:lightwood-2617: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2617:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2617:Featurizing the data
DEBUG:lightwood-2617: `featurize` runtime: 0.0 seconds
INFO:dataprep_ml-2617:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2617:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2617:The block ICP is now running its explain() method
INFO:lightwood-2617:The block ConfStats is now running its explain() method
INFO:lightwood-2617:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2617:The block AccStats is now running its explain() method
INFO:lightwood-2617:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2617:The block PermutationFeatureImportance is now running its explain() method
INFO:l

,original_index,prediction,confidence,lower,upper
0,0,56.392408,0.9991,21.057082,91.727734
1,1,35.078092,0.9991,0.000000,70.413418
2,2,13.415072,0.9991,0.000000,48.750398
3,3,4.565046,0.9991,0.000000,39.900372
4,4,40.870064,0.9991,5.534738,76.205390
...,...,...,...,...,...
201,201,47.530615,0.9991,12.195289,82.865941
202,202,39.329738,0.9991,3.994412,74.665064
203,203,33.443645,0.9991,0.000000,68.778971
204,204,22.981081,0.9991,0.000000,58.316407


## Updating the predictor

For this, we have two options:

### `BaseMixer.partial_fit()`

Updates a single mixer. You need to pass the new data wrapped in `EncodedDs` objects.

**Arguments:** 
* `train_data: EncodedDs`
* `dev_data: EncodedDs`
* `adjust_args: Optional[dict]` - This will contain any arguments needed by the mixer to adjust new data.

If the mixer does not need a `dev_data` partition, pass a dummy:

```
dev_data = EncodedDs(predictor.encoders, pd.DataFrame(), predictor.target)
```

### `PredictorInterface.adjust()`

Updates all mixers inside the predictor by calling their respective `partial_fit()` methods. Any `adjust_args` will be transparently passed as well.

**Arguments:**

* `new_data: pd.DataFrame`
* `old_data: Optional[pd.DataFrame]`
* `adjust_args: Optional[dict]`

Let's `adjust` our predictor:

In [5]:
predictor.adjust(update_df, train_df)  # data to adjust and original data

INFO:dataprep_ml-2617:Cleaning the data
DEBUG:lightwood-2617: `preprocess` runtime: 0.02 seconds
INFO:dataprep_ml-2617:Cleaning the data
DEBUG:lightwood-2617: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2617:Updating the mixers
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
INFO:lightwood-2617:Loss @ epoch 1: 0.021607850057383377
DEBUG:lightwood-2617: `adjust` runtime: 4.32 seconds


In [6]:
new_predictions = predictor.predict(test_df)
new_predictions

INFO:dataprep_ml-2617:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2617:Cleaning the data
DEBUG:lightwood-2617: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2617:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2617:Featurizing the data
DEBUG:lightwood-2617: `featurize` runtime: 0.0 seconds
INFO:dataprep_ml-2617:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2617:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2617:The block ICP is now running its explain() method
INFO:lightwood-2617:The block ConfStats is now running its explain() method
INFO:lightwood-2617:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2617:The block AccStats is now running its explain() method
INFO:lightwood-2617:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2617:The block PermutationFeatureImportance is now running its explain() method
INFO:l

,original_index,prediction,confidence,lower,upper
0,0,56.552303,0.9991,21.216977,91.887629
1,1,36.138442,0.9991,0.803116,71.473768
2,2,13.947593,0.9991,0.000000,49.282919
3,3,5.081975,0.9991,0.000000,40.417301
4,4,41.397517,0.9991,6.062191,76.732843
...,...,...,...,...,...
201,201,47.714752,0.9991,12.379426,83.050078
202,202,39.348288,0.9991,4.012962,74.683614
203,203,33.963747,0.9991,0.000000,69.299073
204,204,23.475346,0.9991,0.000000,58.810672


Nice! Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies to complete the experiment:

In [7]:
from sklearn.metrics import r2_score
import numpy as np

old_acc = r2_score(test_df['concrete_strength'], predictions['prediction'])
new_acc = r2_score(test_df['concrete_strength'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

Old Accuracy: 0.784
New Accuracy: 0.78


## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.